In [ ]:

from bqplot import pyplot as plt
import ipywidgets as widgets
import numpy as np

In [ ]:
# generate some fake 
n = 2000
x = np.linspace(0.0, 10.0, n)
np.random.seed(0)
y = np.cumsum(np.random.randn(n)*10).astype(int)

In [ ]:
fig_hist = plt.figure( title='Histogram')
hist = plt.hist(y, bins=25)

In [ ]:
hist